In [87]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 1 # how many independent sequences will we process in parallel?
block_size = 3 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 2
n_head = 1
n_layer = 1
dropout = 0.2
# ------------

torch.manual_seed(1337)

# Replace the input text with the given sentence
text = "ik ben erg blij"

# Tokenize the text at word level
words = text.split(' ')
unique_words = sorted(list(set(words)))
vocab_size = len(unique_words)

# Create a mapping from words to integers
stoi = {word: i for i, word in enumerate(unique_words)}
itos = {i: word for i, word in enumerate(unique_words)}
encode = lambda s: [stoi[c] for c in s.split(' ')]  # encoder: take a string, output a list of integers
decode = lambda l: ' '.join([itos[i] for i in l])  # decoder: take a list of integers, output a string

# Use the entire dataset as train_data
data = torch.tensor(encode(text), dtype=torch.long)

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y

    if len(data) - block_size <= 0:
        raise ValueError("block_size should be smaller than the total number of tokens in the text")
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [89]:
xb, yb = get_batch('train')

print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension

    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")



inputs:
torch.Size([1, 3])
tensor([[3, 0, 2]], device='cuda:0')
targets:
torch.Size([1, 3])
tensor([[0, 2, 1]], device='cuda:0')
----
when input is [3] the target: 0
when input is [3, 0] the target: 2
when input is [3, 0, 2] the target: 1


In [97]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        print("query")
        print(q)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)

        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        print("out")
        print(wei)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)

        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)


        if targets is None:
            loss = None
        else:
            print("training:")
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            print("logits")
            print(logits)
            targets = targets.view(B*T)
            print("targets")
            print(targets)
            loss = F.cross_entropy(logits, targets)
            print("loss")
            print(loss)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
 
            # get the predictions
            logits, loss = self(idx_cond)
        
            print("before last timestep")
            print(logits)
            
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            print("last timestep")
            print(logits)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)

            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = GPTLanguageModel()
m = model.to(device)


prompt = "ik ben"
encoded_prompt = torch.tensor(encode(prompt), dtype=torch.long, device=device).unsqueeze(0)

# Generate from the model with a maximum of 5 tokens
generated_tokens = m.generate(encoded_prompt, max_new_tokens=1)[0].tolist()

# Decode and print the generated text
print(decode(generated_tokens))

data

query
tensor([[[-0.0110, -0.0181],
         [ 0.0125,  0.0205]]], device='cuda:0', grad_fn=<UnsafeViewBackward0>)
out
tensor([[[1.2500, 0.0000],
         [0.6250, 0.6250]]], device='cuda:0', grad_fn=<NativeDropoutBackward0>)
einde logits
tensor([[[ 0.0086,  0.0058, -0.0034,  0.0150],
         [-0.0093, -0.0062,  0.0036, -0.0162]]], device='cuda:0',
       grad_fn=<ViewBackward0>)
before last timestep
tensor([[[ 0.0086,  0.0058, -0.0034,  0.0150],
         [-0.0093, -0.0062,  0.0036, -0.0162]]], device='cuda:0',
       grad_fn=<ViewBackward0>)
last timestep
tensor([[-0.0093, -0.0062,  0.0036, -0.0162]], device='cuda:0',
       grad_fn=<SliceBackward0>)
ik ben erg


tensor([3, 0, 2, 1])

In [73]:
decode([2, 0, 1,3])
#0 = ben
#1 = blij
#2 = ik


'erg ben blij ik'

In [4]:
#training
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on the train set
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()



step 0: train loss 1.1416
step 500: train loss 0.0067
step 1000: train loss 0.0018
step 1500: train loss 0.0008
step 2000: train loss 0.0004
step 2500: train loss 0.0003
step 3000: train loss 0.0002
step 3500: train loss 0.0001
step 4000: train loss 0.0001
step 4500: train loss 0.0001
step 4999: train loss 0.0000


In [94]:
logits, loss = model(xb, yb)

xb,yb


query
tensor([[[-0.0345, -0.0317],
         [ 0.0390,  0.0359],
         [-0.0060, -0.0055]]], device='cuda:0', grad_fn=<UnsafeViewBackward0>)
out
tensor([[[1.2500, 0.0000, 0.0000],
         [0.6255, 0.6245, 0.0000],
         [0.4166, 0.4167, 0.0000]]], device='cuda:0',
       grad_fn=<NativeDropoutBackward0>)
training:
logits
tensor([[ 0.0246, -0.0237,  0.0045, -0.0116],
        [-0.0258,  0.0249, -0.0047,  0.0121],
        [ 0.0057, -0.0055,  0.0010, -0.0027]], device='cuda:0',
       grad_fn=<ViewBackward0>)
targets
tensor([0, 2, 1], device='cuda:0')
loss
tensor(1.3815, device='cuda:0', grad_fn=<NllLossBackward0>)


(tensor([[3, 0, 2]], device='cuda:0'), tensor([[0, 2, 1]], device='cuda:0'))